```{ojs}
//| output: false
//| echo: false

  data = await FileAttachment("stats_dummyExample.csv").csv({typed: true});
  getFields=function(){
    var availData=[];
      for (var i = 0; i <= data.length - 1; i++) {
        //availData+=' ' + data[i]['effectName'] + ','´
        availData.push(data[i]['effectName'])
      }
      
      return availData//availData.slice(0,-1)
  }
  getStats=function(targetEffect, printCI="false"){
    var outputStr ='('
    var effectdata
      for (var i = 0; i <= data.length - 1; i++) {
        if (data[i]['effectName']==targetEffect){
          effectdata= data[i];
        }
      }
    if (effectdata['pvalue'].toFixed(3)[-1]=='0'){
      var pvalStr=", p="+ effectdata['pvalue'].toFixed(3);
    } else {
      var pvalStr=", p<0.001";
    }
    if ((printCI=="true") && effectdata['lowCI']!==null  && effectdata['highCI']!==null){
      var ciStr=", CI " + effectdata['lowCI'].toFixed(2) + '-' + effectdata['highCI'].toFixed(2);
    } else {
      var ciStr="";
    }
    if (effectdata['stat']=='t'){
      outputStr+='t(' + effectdata['df1'] +  ')=' + effectdata['effectSize'] + pvalStr
        
    } else if (effectdata['stat']=='F'){
      outputStr+='F(' + effectdata['df1'] +  ',' + effectdata['df2'] + ')=' + effectdata['effectSize'].toFixed(1) + pvalStr;
    }
    outputStr += ciStr + ')';
    return outputStr
  }
  'Input data'
  data
  'Effects available in this csv file'
  fields=getFields()
  fields
  'To deactivate these messages, just set output: false in the header'
```



## Using a convoluted function to extract stats from a .csv file

The effect was clearly significant \${getStats("groupdiff","true")}.

In general, .csv files are not optimal for this type of work because we
will usually not work with big matrices when reporting stats.

You can check `_stat_function_csv` if you are curious.

## A smarter approach based on JSON files

JSON files are dominating the internet, but not only. They are very useful to encode complex data structure.

The good thing is that Matlab structure, Python dictionary, Pandas dataframes and many other formats used by scientists can be directly encoded as json.

We can import any json data in quarto like this.


```{ojs}
//| output: false
//| echo: true
  stats = await FileAttachment("data_analysis/stats/stock_ttest.json").json();
```


Then we can extract its information.

```
\${stats["p"].toFixed(4)}
```
gives \${stats["p"].toFixed(4)}

```
\${stats["ci"][0].toFixed(2)}
```
gives \${stats["ci"][0].toFixed(2)}

```
\${stats["ci"][1].toFixed(2)}
```
gives \${stats["ci"][1].toFixed(2)}

```
\${stats["stats"]["tstat"].toFixed(2)}
```
gives \${stats["stats"]["tstat"].toFixed(2)}

```
\${stats["stats"]["df"]}
```
gives \${stats["stats"]["df"]}


So, we can combine all these information and write.

```
(t(\${stats["stats"]["df"]})= \${stats["stats"]["tstat"].toFixed(2)}, p= \${stats["p"].toFixed(4)}, CI= \${stats["ci"][0].toFixed(2)}, \${stats["ci"][1].toFixed(2)})
```
The content of the following parenthesis is generated programmatically by the code above (t(\${stats["stats"]["df"]})= \${stats["stats"]["tstat"].toFixed(2)}, p= \${stats["p"].toFixed(4)}, CI= \${stats["ci"][0].toFixed(2)}, \${stats["ci"][1].toFixed(2)}).

If you are an aficionado, you might have recognized the default naming scheme of Matlab ttest function.

And that indeed the case. These statistics come from the following snippet of code:


```{matlab}
folder_stats_json='stats';
%% t-test example
clear test
load stockreturns
x = stocks(:,3);
[test.h,test.p,test.ci,test.stats] = ttest(x);
% save the test as json
testname='stock_ttest';
encodedStats=jsonencode(test);
fid = fopen([folder_stats_json '\' testname '.json'],'w');
fprintf(fid,'%s',encodedStats);
fclose(fid);
```